In [1]:
import pandas as pd
import json
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
)
from typing import List, Dict
import seaborn as sns
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dataset class

# Load data from json file
with open('../reports/generic_epochs_200_train_size_full.json') as f:
    data = json.load(f)

dfs = []
for k, v in data.items():
    valid_metrics = v['valid']
    valid_metrics['dataset'] = 'valid'
    valid_metrics['fold'] = int(k) + 1
    dfs.append(pd.DataFrame([valid_metrics]))
    
    test_metrics = v['test']
    test_metrics['dataset'] = 'test'
    test_metrics['fold'] = int(k) + 1
    dfs.append(pd.DataFrame([test_metrics]))

# Concatenate all dataframes together
df = pd.concat(dfs, ignore_index=True)

# Rename columns
df.columns = df.columns.str.replace('eval_', '')
df = df.rename(columns={'epoch': 'num_epochs'})

# Print the final dataframe
print(df)

c:\Users\bruno\.conda\envs\my_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


       loss  accuracy  micro_precision  micro_recall  micro_f1  \
0  0.121153       0.0         0.819905      0.800926  0.810304   
1  0.138046       0.0         0.787618      0.772483  0.779977   
2  0.141095       0.0         0.798335      0.784545  0.791380   
3  0.140656       0.0         0.793049      0.771714  0.782236   
4  0.109987       0.0         0.808470      0.786517  0.797342   
5  0.116768       0.0         0.796516      0.773251  0.784711   
6  0.125396       0.0         0.819923      0.794063  0.806786   
7  0.132581       0.0         0.795075      0.769408  0.782031   
8  0.111217       0.0         0.816387      0.773620  0.794428   
9  0.118092       0.0         0.790660      0.754804  0.772316   

   macro_precision  macro_recall  macro_f1  runtime  samples_per_second  \
0         0.785532      0.732025  0.750269    6.767             118.221   
1         0.706886      0.657421  0.674854    8.425             118.694   
2         0.782169      0.704428  0.732035    7.

In [2]:
df

,loss,accuracy,micro_precision,micro_recall,micro_f1,macro_precision,macro_recall,macro_f1,runtime,samples_per_second,steps_per_second,num_epochs,dataset,fold
0,0.121153,0.0,0.819905,0.800926,0.810304,0.785532,0.732025,0.750269,6.767,118.221,29.555,7.5,valid,1
1,0.138046,0.0,0.787618,0.772483,0.779977,0.706886,0.657421,0.674854,8.425,118.694,29.674,7.5,test,1
2,0.141095,0.0,0.798335,0.784545,0.791380,0.782169,0.704428,0.732035,7.043,113.588,28.397,8.0,valid,2
3,0.140656,0.0,0.793049,0.771714,0.782236,0.714138,0.672311,0.687991,8.301,120.467,30.117,8.0,test,2
4,0.109987,0.0,0.808470,0.786517,0.797342,0.778960,0.636299,0.681311,6.625,120.755,30.189,5.0,valid,3
5,0.116768,0.0,0.796516,0.773251,0.784711,0.760337,0.651862,0.685468,8.277,120.817,30.204,5.0,test,3
6,0.125396,0.0,0.819923,0.794063,0.806786,0.797461,0.705027,0.738831,6.962,114.910,28.727,7.5,valid,4
7,0.132581,0.0,0.795075,0.769408,0.782031,0.716366,0.663523,0.685407,8.477,117.966,29.492,7.5,test,4
8,0.111217,0.0,0.816387,0.773620,0.794428,0.741366,0.679100,0.693930,7.301,109.574,27.394,5.0,valid,5
9,0.118092,0.0,0.790660,0.754804,0.772316,0.750471,0.657315,0.688811,9.509,105.164,26.291,5.0,test,5


In [1]:
def model_summary(model):
    print("Model summary:")
    print("---------------------------")
    total_params = 0
    for name, param in model.named_parameters():
        param_count = param.numel()
        total_params += param_count
    print(f"Total parameters: {total_params}")
    
"""def print_report(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=categories)
    print(report)
    sns.heatmap(cm, annot=True, xticklabels=categories, yticklabels=categories, fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()"""

class TweetDataset(Dataset):
    def __init__(self, x, y, mlb, tokenizer):
        self.x = x
        self.y = y
        self.mlb = mlb
        self.tokenizer = tokenizer
        self.encoded_tweets = self.preprocess_text(self.x)
    
    def preprocess_text(self, text):
        return self.tokenizer(text, return_attention_mask=True, return_tensors='pt', padding=True)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        label = self.y[idx]
        return {'input_ids': self.encoded_tweets['input_ids'][idx],
                'attention_mask': self.encoded_tweets['attention_mask'][idx],
                'label': torch.tensor(label, dtype=torch.float32)}
        
class MultiLabelDataCollator(DataCollatorWithPadding):
    def __init__(self, tokenizer):
        super().__init__(tokenizer)

    def __call__(self, features: List[Dict[str, torch.Tensor]]):
        batch = super().__call__(features)
        batch["labels"] = torch.stack([feature["label"] for feature in features])
        return batch
    
def get_classification_report(data_loader, model, target_names, label_names):
    labels = []
    predictions = []
    for batch in data_loader:
        batch_inputs = {'input_ids': batch['input_ids'].to(device),
                        'attention_mask': batch['attention_mask'].to(device)}
        with torch.no_grad():
            logits = model(**batch_inputs).logits
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(torch.Tensor(logits))
        batch_predictions = (probs >= 0.5).detach().cpu().numpy().astype(int)
        
        predictions.append(batch_predictions)
        labels.append(batch['labels'].detach().cpu().numpy().astype(int))

    predictions = np.concatenate(predictions, axis=0)
    labels = np.concatenate(labels, axis = 0)

    #cm = multilabel_confusion_matrix(labels, predictions)
    dict_report = classification_report(labels, predictions, target_names=target_names, labels=label_names, zero_division=0, output_dict=True)
    report = classification_report(labels, predictions, target_names=target_names, labels=label_names, zero_division=0)
    return dict_report, report
    
def calculate_average_report(reports):
    avg_report = {}
    for report in reports:
        for key, scores in report.items():
            if key not in avg_report:
                avg_report[key] = {}
                for score_key, score_value in scores.items():
                    avg_report[key][score_key] = score_value
            else:
                for score_key, score_value in scores.items():
                    avg_report[key][score_key] += score_value

    num_reports = len(reports)
    for key, scores in avg_report.items():
        for score_key in scores:
            avg_report[key][score_key] /= num_reports

    return avg_report

def average_report_to_dataframe(average_report):
    data = {
        "precision": [],
        "recall": [],
        "f1-score": [],
        "support": []
    }
    index = []

    for class_name, metrics in average_report.items():
        if class_name == 'accuracy':
            continue

        index.append(class_name)
        data["precision"].append(metrics["precision"])
        data["recall"].append(metrics["recall"])
        data["f1-score"].append(metrics["f1-score"])
        data["support"].append(metrics["support"])

    return pd.DataFrame(data, index=index)

def calculate_metrics(task):
    k = 5
    
    val_classification_reports = []
    test_classification_reports = []

    # Loop over each fold and load the corresponding model
    for fold in range(k):
        model_path = f"../models/{task}_epochs_200_train_size_full_fold_{fold}"
        # find the latest checkpoint file
        #checkpoint_files = [f for f in os.listdir(model_path) if f.startswith("checkpoint")]
        latest_checkpoint = os.path.join(model_path, "")  # use "" for models that were manually saved after training. use sorted(checkpoint_files)[0] for the first automatically saved checkpoint 
        print(latest_checkpoint)
        
        # Load the model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(latest_checkpoint)
        model.to(device)
        tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

        filename = f"../data/labeled_data/{task}_test_{fold}.json"
        with open(filename) as f:
            data = json.load(f)
        train_df = pd.DataFrame(data["train"])
        val_df = pd.DataFrame(data["valid"])
        test_df = pd.DataFrame(data["test"])
        
        train_annotations = train_df["annotations"].tolist()
        classes = set()
        for annotation in train_annotations:
            classes.update(annotation)
        classes = sorted(list(classes))
        
        checkpoint = torch.load(os.path.join(model_path, "pytorch_model.bin"))
        model.load_state_dict(checkpoint)
        
        mlb = MultiLabelBinarizer(classes=classes)
        
        train_labels = mlb.fit_transform(train_df["annotations"])
        val_labels = mlb.transform(val_df["annotations"])
        test_labels = mlb.transform(test_df["annotations"])
        
        train_dataset = TweetDataset(train_df['text'].to_list(), torch.tensor(train_labels), mlb, tokenizer)
        val_dataset = TweetDataset(val_df['text'].to_list(), torch.tensor(val_labels), mlb, tokenizer)
        test_dataset = TweetDataset(test_df['text'].to_list(), torch.tensor(test_labels), mlb, tokenizer)
        
        val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=4, shuffle=False, collate_fn=MultiLabelDataCollator(tokenizer)
        )
        test_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=4, shuffle=False, collate_fn=MultiLabelDataCollator(tokenizer)
        )
        
        model.eval()
        val_report_dict, val_report = get_classification_report(val_loader, model, classes, range(len(classes)))
        test_report_dict, test_report = get_classification_report(test_loader, model, classes, range(len(classes)))
        val_classification_reports.append(val_report_dict)
        test_classification_reports.append(test_report_dict)

    val_average_report = calculate_average_report(val_classification_reports)
    test_average_report = calculate_average_report(test_classification_reports)
    val_average_report_df = average_report_to_dataframe(val_average_report)
    test_average_report_df = average_report_to_dataframe(test_average_report)
    print("\nAverage Validation Classification Report In DataFrame Format:")
    print(val_average_report_df) 
    print("\nAverage Test Classification Report In DataFrame Format:")
    print(test_average_report_df) 
    return val_average_report_df, test_average_report_df

generic_val_average_report_df, generic_test_average_report_df = calculate_metrics("generic")
GRU_202012_val_average_report_df, GRU_202012_test_average_report_df = calculate_metrics("GRU_202012")
IRA_202012_val_average_report_df, IRA_202012_test_average_report_df = calculate_metrics("IRA_202012")
REA_0621_val_average_report_df, REA_0621_test_average_report_df = calculate_metrics("REA_0621")
UGANDA_0621_val_average_report_df, UGANDA_0621_test_average_report_df = calculate_metrics("UGANDA_0621")
VENEZUELA_201901_2_val_average_report_df, VENEZUELA_201901_2_test_average_report_df = calculate_metrics("VENEZUELA_201901_2")

NameError: name 'Dataset' is not defined

In [10]:
import pandas as pd

dataframes = {
    "generic_val_average_report": generic_val_average_report_df,
    "generic_test_average_report": generic_test_average_report_df,
    "GRU_202012_val_average_report": GRU_202012_val_average_report_df,
    "GRU_202012_test_average_report": GRU_202012_test_average_report_df,
    "IRA_202012_val_average_report": IRA_202012_val_average_report_df,
    "IRA_202012_test_average_report": IRA_202012_test_average_report_df,
    "REA_0621_val_average_report": REA_0621_val_average_report_df,
    "REA_0621_test_average_report": REA_0621_test_average_report_df,
    "UGANDA_0621_val_average_report": UGANDA_0621_val_average_report_df,
    "UGANDA_0621_test_average_report": UGANDA_0621_test_average_report_df,
    "VENEZUELA_201901_2_val_average_report": VENEZUELA_201901_2_val_average_report_df,
    "VENEZUELA_201901_2_test_average_report": VENEZUELA_201901_2_test_average_report_df,
}

for name, df in dataframes.items():
    csv_filename = f"../reports/{name}_normalized_tweets.csv"
    df.to_csv(csv_filename, index=False)

In [18]:
generic_val_average_report_df[generic_val_average_report_df.index == "macro avg"]["f1-score"].values[0]

0.7053414707928204

# Macro Averages:

In [11]:
def extract_macro_avg_value(df):
    return df[df.index == "macro avg"]["f1-score"].values[0]

summary_data = []

for name, df in dataframes.items():
    micro_avg_value = round(extract_macro_avg_value(df), 2)
    train_data = name.split("_")[0]

    if "val" in name:
        validation_value = micro_avg_value
        test_value = None
    elif "test" in name:
        validation_value = None
        test_value = micro_avg_value

    test_data = train_data
    if train_data != "generic":
        train_data = "All but " + train_data

    summary_data.append({
        "Train Data": train_data,
        "Test Data": test_data,
        "Validation": validation_value,
        "Test": test_value,
    })

# Combine rows with the same "Train Data" and "Test Data" into one
macro_summary_df = pd.DataFrame(summary_data)
macro_summary_df = macro_summary_df.groupby(["Train Data", "Test Data"], as_index=False).first()

# Reorder columns
macro_summary_df = macro_summary_df[["Train Data", "Test Data", "Validation", "Test"]]
macro_summary_df = macro_summary_df.reindex([macro_summary_df.index[-1]] + list(macro_summary_df.index[:-1]))
macro_summary_df = macro_summary_df.reset_index(drop=True)

print(macro_summary_df)

          Train Data  Test Data  Validation  Test
0            generic    generic        0.71  0.67
1        All but GRU        GRU        0.67  0.44
2        All but IRA        IRA        0.69  0.55
3        All but REA        REA        0.69  0.55
4     All but UGANDA     UGANDA        0.71  0.48
5  All but VENEZUELA  VENEZUELA        0.66  0.59


# Micro Averages:

In [21]:
def extract_micro_avg_value(df):
    print("no values: ", df[df.index == "micro avg"]["f1-score"])
    print("wotjvalies : ", df[df.index == "micro avg"]["f1-score"].values[0])
    return df[df.index == "micro avg"]["f1-score"].values[0]

summary_data = []

for name, df in dataframes.items():
    micro_avg_value = round(extract_micro_avg_value(df), 2)
    train_data = name.split("_")[0]

    if "val" in name:
        validation_value = micro_avg_value
        test_value = None
    elif "test" in name:
        validation_value = None
        test_value = micro_avg_value

    test_data = train_data
    if train_data != "generic":
        train_data = "All but " + train_data

    summary_data.append({
        "Train Data": train_data,
        "Test Data": test_data,
        "Validation": validation_value,
        "Test": test_value,
    })

# Combine rows with the same "Train Data" and "Test Data" into one
micro_summary_df = pd.DataFrame(summary_data)
micro_summary_df = micro_summary_df.groupby(["Train Data", "Test Data"], as_index=False).first()

# Reorder columns
micro_summary_df = micro_summary_df[["Train Data", "Test Data", "Validation", "Test"]]
micro_summary_df = micro_summary_df.reindex([micro_summary_df.index[-1]] + list(micro_summary_df.index[:-1]))
micro_summary_df = micro_summary_df.reset_index(drop=True)

print(micro_summary_df)

no values:  micro avg    0.79461
Name: f1-score, dtype: float64
wotjvalies :  0.7946103414885577
no values:  micro avg    0.776106
Name: f1-score, dtype: float64
wotjvalies :  0.776105954235275
no values:  micro avg    0.776579
Name: f1-score, dtype: float64
wotjvalies :  0.7765792870382628
no values:  micro avg    0.773844
Name: f1-score, dtype: float64
wotjvalies :  0.7738437287750406
no values:  micro avg    0.814075
Name: f1-score, dtype: float64
wotjvalies :  0.8140749682464591
no values:  micro avg    0.629581
Name: f1-score, dtype: float64
wotjvalies :  0.6295805199879053
no values:  micro avg    0.796831
Name: f1-score, dtype: float64
wotjvalies :  0.7968310520875482
no values:  micro avg    0.722081
Name: f1-score, dtype: float64
wotjvalies :  0.7220808010471017
no values:  micro avg    0.800973
Name: f1-score, dtype: float64
wotjvalies :  0.8009728728839356
no values:  micro avg    0.760924
Name: f1-score, dtype: float64
wotjvalies :  0.7609242914494959
no values:  micro avg 